# Polynomial expansion of inputs

In this notebook, we introduce the `PolynomialExpansion` module which is able to create polynomial features of inputs in a differentiable way.

In [ ]:
import os
import sys

sys.path.append(os.path.join(os.path.abspath(''), ".."))

import torch

from nnbma.networks import FullyConnected, PolynomialNetwork
from nnbma.layers import PolynomialExpansion

## `PolynomialExpansion` module

`PolynomialExpansion` is a torch `Module` that creates all possible (non-constant) monomial from a set of inputs. For instance, for `degree=2`, we have:

$ \mathrm{poly}((x_1,\,x_2,\,x_3)) = (x_1,\,x_2,\,x_3,\,x_1^2,\,x_1x_2,\,x_1x_3,\,x_2^2,\,x_2x_3,\,x_3^2) $.

Here's the corresponding expansion:

In [ ]:
input_features = 3
order = 2

layer = PolynomialExpansion(input_features, order, standardize=False)

x = torch.tensor([2., 3., 5.]) # Must have x.shape[-1] = input_features
print("Input:", x)

y = layer(x)
print("Output:", y)

As any modules from this package, it works with batched inputs along the first axes:

In [ ]:
x = torch.tensor([
    [
        [2., 3., 5.],
        [1., 1., 1.],
    ], [
        [1., 0., 0.],
        [0., 2., 3.],
    ]
]) # Must have x.shape[-1] = input_features
print("Input:", x)

y = layer(x)
print("Output:", y)

Contrary to its classical use in preprocessing, this expansion is completely differentiable with respect to its inputs, so that it can be integrated into a neural network (and not placed before, a situation where the derivation of the network with respect to the inputs would be performed with respect to the developed inputs, and not with respect to the real inputs).

In [ ]:
x = torch.tensor([2., 3., 5.], requires_grad=True) # Must have x.shape[-1] = input_features
print("Input:", x)

y = layer(x)
print("Output gradient:", y)

## `PolynomialNetwork` module

`PolynomialNetwork` is a convenience class that allows to integrate a `PolynomialLayer` at the input of a network inheriting from `NeuralNetwork`.

In [ ]:
subnet = FullyConnected(
    [PolynomialExpansion.expanded_features(order, input_features), 10, 10, 1], # expanded_features allow to anticipate the number of polynomial features that the subnetwork will have as input, depending on the number of real input features and the max order.
    torch.nn.ReLU(),
)

net = PolynomialNetwork(
    input_features,
    order,
    subnet,
)
net.eval()

In [ ]:
x = torch.tensor([2., 3., 5.], requires_grad=True) # Must have x.shape[-1] = input_features
print("Input:", x)

y = net(x)
print("Output gradient:", y)